# Анализ данных онлайн магазина чтения книг по подписке

## Посчитать, сколько книг вышло после 1 января 2000 года;


In [7]:
query = '''
SELECT 
    COUNT(book_id)
FROM 
    books
WHERE 
    publication_date > '2000-01-01';

'''

In [8]:
pd.io.sql.read_sql(query, con = engine)

,count
0,819


<b>Вывод </b>  
В базе данных имеется информация о 819 книгах, вышедших после 1 января 2000 года

## Для каждой книги посчитать количество обзоров и среднюю оценку;


In [9]:
query = '''
WITH

avg_rating AS 
    (SELECT book_id, 
    ROUND(AVG(RATING),2) AS avg_rat
    FROM ratings
    GROUP BY book_id
    ORDER BY book_id),

count_reviews AS 
    (SELECT book_id, 
    COUNT(review_id) AS count_rev
    FROM reviews
    GROUP BY book_id
    ORDER BY book_id)

SELECT 
    b.title,
    b.book_id,
    rat.avg_rat,
    rev.count_rev

FROM 
    books AS b 
FULL JOIN --- используем оператор FULL JOIN чтобы в запрос попали все книги, в т.ч. те, которые не получили оценок или ревью 
    avg_rating AS rat ON b.book_id = rat.book_id
FULL JOIN count_reviews AS rev ON b.book_id = rev.book_id


ORDER BY
    rat.avg_rat DESC,
    rev.count_rev DESC
    

LIMIT 
    10;
'''

pd.io.sql.read_sql(query, con = engine)

,title,book_id,avg_rat,count_rev
0,The Cat in the Hat and Other Dr. Seuss Favorites,672,5.0,NaN
1,A Dirty Job (Grim Reaper #1),17,5.0,4.0
2,Moneyball: The Art of Winning an Unfair Game,444,5.0,3.0
3,School's Out—Forever (Maximum Ride #2),553,5.0,3.0
4,Captivating: Unveiling the Mystery of a Woman'...,136,5.0,2.0
5,Alas Babylon,62,5.0,2.0
6,Crucial Conversations: Tools for Talking When ...,169,5.0,2.0
7,Act of Treason (Mitch Rapp #9),57,5.0,2.0
8,A Fistful of Charms (The Hollows #4),20,5.0,2.0
9,A Woman of Substance (Emma Harte Saga #1),55,5.0,2.0


<b>Вывод </b>  
В соответствии с запросом составлена таблица со количеством ревью и средним рейтингом. Лидер по рейтингу и по ревью - книга `A Dirty Job (Grim Reaper #1)`. У нее 5 баллов и 4 обзора. Следует отметить, что на  книгу `The Cat in the Hat and Other Dr. Seuss Favorites` не оставлено ни одного обзора не смотря на ее высокий рейтинг

## Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (исключение брошюр)


In [10]:
query = '''

WITH 
cnt_of_books AS
        (SELECT 
        publisher_id,
        COUNT(DISTINCT book_id) AS count_of_books
        FROM books
        WHERE  num_pages > 50 
        GROUP BY publisher_id)
        

SELECT 
    p.publisher,
    p.publisher_id,
    c.count_of_books
FROM
    publishers AS p 
    JOIN cnt_of_books AS c ON p.publisher_id = c.publisher_id

ORDER BY 
    c.count_of_books DESC
LIMIT 1;

'''
pd.io.sql.read_sql(query, con = engine)

,publisher,publisher_id,count_of_books
0,Penguin Books,212,42


<b>Вывод </b>  
Издательсво, которое выпустило больше всего книг - `Penguin Books`.     
За весь период данное издательство выпустило 42 книги больше 50 страниц

## Определить автора с самой высокой средней оценкой книг — будем учитывать только книги с 50 и более оценками;


In [11]:
query = '''

SELECT
    t.author,
    SUM(t.rating_count) AS number_of_ratings,
    ROUND(AVG(t.avg_rating),2) AS overall_rating

FROM
    (SELECT 
        r.book_id AS book_id,
        b.title AS title,
        a.author AS author,
        COUNT(r.rating_id) AS rating_count,
        AVG(r.rating) AS avg_rating
    FROM 
        ratings AS r
        LEFT JOIN books as b ON r.book_id = b.book_id
        LEFT JOIN authors AS a ON b.author_id = a.author_id
    GROUP BY 
        1,
        2,
        3
    HAVING 
        COUNT(r.rating_id) > 50) AS t

GROUP BY 
    1
ORDER BY 
    3 DESC,
    2 DESC
LIMIT
    1;

        '''
pd.io.sql.read_sql(query, con = engine)
        

,author,number_of_ratings,overall_rating
0,J.K. Rowling/Mary GrandPré,310.0,4.28


<b>Вывод </b>  
Автор, имеющий наибольшее среднее значение рейтинга книг, для книг с количеством оценок не менее 50 - это `J.K. Rowling/Mary GrandPré`. Книги этого автора собрали 310 оценок и среднее занчение рейтинга составляет 4.28 баллов из 5

## Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
query = '''


SELECT 
    COUNT(a.username) AS num_of_users,
    ROUND(AVG(num_of_reviews),2) AS avg_reviews
FROM
    (SELECT 
        rat.username,
        COUNT(rat.rating_id) AS num_of_marks
    FROM 
        ratings AS rat 
    GROUP BY 
        rat.username
    HAVING 
        COUNT(rat.rating_id) > 50) AS a

LEFT JOIN 

    (SELECT 
        username,
        count(review_id) AS num_of_reviews
    FROM 
        reviews 
    GROUP BY 1) AS b 
    
ON a.username = b.username



'''
pd.io.sql.read_sql(query, con = engine)

,num_of_users,avg_reviews
0,6,24.33


<b>Вывод </b>  
Всего 6 пользователей сервиса оставили более 50 оценок к книгам. В среднем эти пользователи написали 24.33 обзора на литературу, представленную на сайте

# Общий вывод
Проведен анализ данных сервиса чтения книг по подписке. 
Получены следующие результаты:
- В базе данных имеется информация о 819 книгах, вышедших после 1 января 2000 года.
- Лидер по рейтингу и по ревью - книга `A Dirty Job (Grim Reaper #1)`. У нее 5 баллов и 4 обзора. 
- Издательсво, которое выпустило больше всего книг - `Penguin Books`. За весь период данное издательство выпустило 42 книги больше 50 страниц
- Автор, имеющий наибольшее среднее значение рейтинга книг, для книг с количеством оценок не менее 50 - это J.K. Rowling/Mary GrandPré.
- Всего 6 пользователей сервиса оставили более 50 оценок к книгам. В среднем эти пользователи написали 24.33 обзора на литературу, представленную на сайте